In [1]:
import os, sys, timeit

import numpy as np
import pandas as pd

from metrics import process

from tqdm import tqdm
from os.path import join
from kmeans_clustering import KMeansClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
pwd = os.getcwd()
data_dir = join(pwd, 'data/')
need_normalization = False
need_standardization = False
need_pca = False
filename = 'mall.csv'
datapath = join(data_dir, filename)
num_clusters = 2
num_iterations = 300
num_iter_exp = 20
verbose = False

In [3]:
# f = open('data/data.csv')
# lines = f.read()
# print(type(lines))
# new_lines = ''
# for line in lines:
#     new_lines += str(line.encode('utf-8').strip())
# print(len(new_lines))

In [4]:
df = pd.read_csv(
    datapath, 
    low_memory=False,
    encoding='utf-8',
#     sep=',',
    header=None,
)

In [5]:
print(len(df))

201


In [6]:
print(df.head())

            0       1    2                   3                       4
0  CustomerID   Genre  Age  Annual Income (k$)  Spending Score (1-100)
1        0001    Male   19                  15                      39
2        0002    Male   21                  15                      81
3        0003  Female   20                  16                       6
4        0004  Female   23                  16                      77


In [7]:
print(df.dtypes)

0    object
1    object
2    object
3    object
4    object
dtype: object


In [8]:
columns = df.columns.tolist()
cols = columns
# cols = columns[1:]
# cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Fare']
# cols = columns[:-1]
# cols = ['Passenger Count', 'Adjusted Passenger Count']
# cols = columns[1:]
cols = columns[2:]
df = df[cols]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)
df = df.replace([-np.inf, np.inf], np.nan)
df = df.dropna()
df = df.loc[:, (df != 0).any(axis=0)]
print(len(df))
print(df.dtypes)

201
2    float64
3    float64
4    float64
dtype: object


In [9]:
df = df.sample(
    frac=1.0,
    random_state=1,
)
df = df.reset_index(drop=True)

In [10]:
if need_normalization is True:
    normalizer = StandardScaler()
    tmp = normalizer.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)
elif need_standardization is True:
    scaler = MinMaxScaler()
    tmp = scaler.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)

In [11]:
X = df
if need_pca is True:
    pca = PCA(
        n_components=len(cols),
        svd_solver='auto',
    )
    X = pca.fit_transform(X)
else:
    X = X.to_numpy()

In [12]:
for x in X:
    for a in x:
        if np.isinf(a):
            print(x)

In [13]:
data = X
print(data[:5])
print(data.shape)

[[47. 60. 47.]
 [63. 65. 43.]
 [27. 46. 51.]
 [49. 33. 14.]
 [50. 85. 26.]]
(201, 3)


In [14]:
models = ['coc', 'kmeans++', 'ostrovsky', 'kmeans']
# models = ['kmeans++', 'kmeans++_improved']

In [15]:
def plot_scatter(X, labels):
    fig, ax = plt.subplots()
    scatter = plt.scatter(
        X[:, 0],
        X[:, 1],
        c=labels,
    )
    handles, labels = scatter.legend_elements()
    legend = ax.legend(
        handles,
        labels,
        loc='upper right',
        title='Label',
    )
    ax.add_artist(legend)
    plt.title(model+' '+str(num_clusters)+ ' clusters')
    plt.xlabel('Component A')
    plt.ylabel('Component B')
    plt.show()

In [16]:
algorithms = {}
iterations = {}
inertias = {}
times = {}
inertias_min = {}
iters_min = {}
for model in models:
    iterations[model] = 0.0
    inertias[model] = 0.0
    times[model] = 0.0
    iters_min[model] = 0
    inertias_min[model] = 1e20
for i in tqdm(range(num_iter_exp)):
    for model in (models):
        algorithms[model] = KMeansClustering(
            n_clusters=num_clusters,
            max_iter=num_iterations,
            init=model,
            verbose=verbose,
        )
        start = timeit.default_timer()
        algorithm = algorithms[model].fit(data)
        centers = algorithm.cluster_centers_
        labels = algorithms[model].predict(data)
#         plot_scatter(X, labels)
        inertias[model] += algorithm.best_inertia_
        inertias_min[model] = min(inertias_min[model], algorithm.best_inertia_)
        iters_min[model] += algorithm.iter_convergence_
        end = timeit.default_timer()
        times[model] += (end-start)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:38<00:00,  1.92s/it]


In [17]:
for model in models:
    print(model+', '+str(num_clusters))
    print('Average Inertia -', round(inertias[model]/num_iter_exp, 2))
    print('Minimum Inertia - ', round(inertias_min[model], 2))
    print('Time -', round(times[model]/num_iter_exp, 2))
#     print('Average number of iterations', round(iters_min[model]/num_iter_exp,2))
    print('')

coc, 2
Average Inertia - 224139.07
Minimum Inertia -  219519.1
Time - 0.49

kmeans++, 2
Average Inertia - 227194.03
Minimum Inertia -  219519.1
Time - 0.48

ostrovsky, 2
Average Inertia - 224401.2
Minimum Inertia -  219519.1
Time - 0.48

kmeans, 2
Average Inertia - 223901.32
Minimum Inertia -  219519.1
Time - 0.48

